In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 1) Training by using CNN-LSTM via images

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import TimeDistributed, Reshape, LSTM
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import os
from google.colab import drive

# Set random seed for reproducibility
tf.keras.utils.set_random_seed(492)

# Load data from Google Drive
def import_data(base_path, class_names, img_height=224, img_width=224):
    images = []
    labels = []
    for class_idx, class_name in enumerate(class_names):
        class_folder = os.path.join(base_path, class_name)
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            try:
                img = load_img(img_path, target_size=(img_height, img_width))
                img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
                images.append(img_array)
                labels.append(class_idx)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    return np.array(images), np.array(labels)

# Define CNN model
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Reshape((54, 54*64)),
        LSTM(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    # Compile with Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    return model

# Main
base_path = '/content/drive/My Drive/mbpose/holdingSquat'
class_names = ['Incorrect_low', 'Correct', 'Incorrect_high']
img_height, img_width = 224, 224

# Load data
x, y = import_data(base_path, class_names, img_height, img_width)
num_classes = len(np.unique(y))

# Create 6 stratified folds
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
folds = list(skf.split(x, y))
train_val_folds = folds[:5]
test_fold = folds[5]

precision_list, recall_list, accuracy_list, f1_list = [], [], [], []

# 5-Fold Cross-Validation
for fold_index, (train_index, val_index) in enumerate(train_val_folds, start=1):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    print(f"\n=== Fold {fold_index} ===")
    print("Class distribution:", pd.Series(y_val).value_counts().to_dict())

    # Class weights for imbalance
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: class_weights[i] for i in range(len(class_weights))}

    # Create model
    model = create_cnn_lstm_model(input_shape=(img_height, img_width, 3), num_classes=num_classes)

    # Early stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train
    model.fit(
        x_train, y_train, epochs=30, batch_size=32, validation_data=(x_val, y_val),
        verbose=1, class_weight=class_weights, callbacks=[early_stop]
    )

    # Evaluate
    predictions = np.argmax(model.predict(x_val), axis=1)
    precision = precision_score(y_val, predictions, average='macro', zero_division=0)
    recall = recall_score(y_val, predictions, average='macro', zero_division=0)
    accuracy = accuracy_score(y_val, predictions)
    f1 = f1_score(y_val, predictions, average='macro', zero_division=0)

    print("Confusion Matrix:")
    print(confusion_matrix(y_val, predictions))
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    f1_list.append(f1)

# Report average scores
print("\n=== 5-Fold Cross-Validation Summary ===")
print(f"Avg Precision: {np.mean(precision_list):.4f}")
print(f"Avg Recall:    {np.mean(recall_list):.4f}")
print(f"Avg Accuracy:  {np.mean(accuracy_list):.4f}")
print(f"Avg F1 Score:  {np.mean(f1_list):.4f}")

# === Final Testing on Fold 6 ===
train_index = np.concatenate([f[0] for f in train_val_folds])
test_index = test_fold[1]
x_train, x_test = x[train_index], x[test_index]
y_train, y_test = y[train_index], y[test_index]

# Class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Train final model
final_model = create_cnn_lstm_model(input_shape=(img_height, img_width, 3), num_classes=num_classes)
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

final_model.fit(
    x_train, y_train, epochs=30, batch_size=32, validation_split=0.1,
    class_weight=class_weights, callbacks=[early_stop], verbose=1
)

# Final test evaluation
test_predictions = np.argmax(final_model.predict(x_test), axis=1)
test_accuracy = accuracy_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_predictions, average='macro', zero_division=0)
test_precision = precision_score(y_test, test_predictions, average='macro', zero_division=0)
test_recall = recall_score(y_test, test_predictions, average='macro', zero_division=0)

print("\n=== Final Test Fold (Fold 6) Evaluation ===")
print("Confusion Matrix:")
print(confusion_matrix(y_test, test_predictions))
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1 Score:  {test_f1:.4f}")


=== Fold 1 ===
Class distribution: {0: 51, 1: 49, 2: 46}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.2993 - loss: 1.1359 - val_accuracy: 0.3562 - val_loss: 1.0963
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 63s 3s/step - accuracy: 0.3330 - loss: 1.0992 - val_accuracy: 0.3973 - val_loss: 1.0963
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 0.3696 - loss: 1.0849 - val_accuracy: 0.4452 - val_loss: 1.0472
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.4853 - loss: 1.0444 - val_accuracy: 0.6644 - val_loss: 0.8431
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.6382 - loss: 0.8618 - val_accuracy: 0.7192 - val_loss: 0.6502
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.7554 - loss: 0.6381 - val_accuracy: 0.8082 - val_loss: 0.4955
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.8599 - loss: 0.3978 - val_accuracy: 0.8836 - val_loss: 0.3547
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.9394 - loss: 0.2082 - val_accuracy: 0.8836 - val_loss:

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.3647 - loss: 1.1062 - val_accuracy: 0.3493 - val_loss: 1.1348
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.3074 - loss: 1.1245 - val_accuracy: 0.3699 - val_loss: 1.0979
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.3872 - loss: 1.0887 - val_accuracy: 0.4247 - val_loss: 1.1039
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.4453 - loss: 1.0626 - val_accuracy: 0.4589 - val_loss: 1.0399
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - accuracy: 0.5492 - loss: 0.9610 - val_accuracy: 0.6986 - val_loss: 0.7560
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.7217 - loss: 0.7185 - val_accuracy: 0.7671 - val_loss: 0.5964
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.8084 - loss: 0.5303 - val_accuracy: 0.8836 - val_loss: 0.3522
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.8926 - loss: 0.3515 - val_accuracy: 0.8836 - val_loss:

...

Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - accuracy: 0.3332 - loss: 1.1205 - val_accuracy: 0.3493 - val_loss: 1.0978
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 0.3022 - loss: 1.1089 - val_accuracy: 0.4384 - val_loss: 1.0911
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.3668 - loss: 1.0916 - val_accuracy: 0.5068 - val_loss: 1.0569
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.4634 - loss: 1.0336 - val_accuracy: 0.6507 - val_loss: 0.8140
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.5974 - loss: 0.8659 - val_accuracy: 0.6849 - val_loss: 0.6914
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 0.7304 - loss: 0.7107 - val_accuracy: 0.7329 - val_loss: 0.6048
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.8116 - loss: 0.5229 - val_accuracy: 0.9178 - val_loss: 0.2398
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.9083 - loss: 0.2842 - val_accuracy: 0.9247 - val_loss:

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.3388 - loss: 1.1255 - val_accuracy: 0.3219 - val_loss: 1.1005
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.3073 - loss: 1.1159 - val_accuracy: 0.3493 - val_loss: 1.0971
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.3503 - loss: 1.0984 - val_accuracy: 0.3699 - val_loss: 1.0871
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.4485 - loss: 1.0801 - val_accuracy: 0.4110 - val_loss: 1.0619
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.4399 - loss: 1.0527 - val_accuracy: 0.5274 - val_loss: 0.9893
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 0.5782 - loss: 0.8936 - val_accuracy: 0.5616 - val_loss: 0.8622
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 0.7187 - loss: 0.7142 - val_accuracy: 0.7260 - val_loss: 0.6783
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.8379 - loss: 0.4345 - val_accuracy: 0.8151 - val_loss:

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 274s 3s/step - accuracy: 0.3801 - loss: 1.0946 - val_accuracy: 0.2521 - val_loss: 1.0910
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 261s 3s/step - accuracy: 0.5222 - loss: 0.9439 - val_accuracy: 0.9096 - val_loss: 0.2946
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 259s 3s/step - accuracy: 0.9199 - loss: 0.2547 - val_accuracy: 0.9808 - val_loss: 0.0734
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 257s 3s/step - accuracy: 0.9820 - loss: 0.0792 - val_accuracy: 0.9863 - val_loss: 0.0429
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 258s 3s/step - accuracy: 0.9922 - loss: 0.0289 - val_accuracy: 0.9945 - val_loss: 0.0133
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 258s 3s/step - accuracy: 0.9992 - loss: 0.0053 - val_accuracy: 0.9945 - val_loss: 0.0084
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 261s 3s/step - accuracy: 0.9993 - loss: 0.0032 - val_accuracy: 0.9945 - val_loss: 0.0072
Epoch 8/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 270s 3s/step - accuracy: 0.9993 - loss: 0.0023 - val_accu

(2) Training by using CNN-LSTM via keypoints

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Set random seed for reproducibility
tf.keras.utils.set_random_seed(492)

# Load and return features and labels
def import_data():
    data = pd.read_csv('mergedshuffledHoldingSquatV2keypoints.csv')
    x = data.iloc[:, :-1].values
    y = data['correct'].astype(int).values
    y = np.searchsorted(np.unique(y), y)  # Make sure y is 0-based index
    return x, y

# Define the CNN-LSTM model
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Main
x, y = import_data()
num_classes = len(np.unique(y))

#6 stratified folds
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
folds = list(skf.split(x, y))
train_val_folds = folds[:5]
test_fold = folds[5]

precision_list, recall_list, accuracy_list, f1_list = [], [], [], []

# 5-Fold Cross-Validation
for fold_index, (train_index, val_index) in enumerate(train_val_folds, start=1):
    x_train_raw, x_val_raw = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Fit scaler only on training data
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train_raw)
    x_val = scaler.transform(x_val_raw)

    # Reshape for CNN input
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

    # Class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: class_weights[i] for i in range(len(class_weights))}

    print(f"\n=== Fold {fold_index} ===")
    print("Class distribution:", pd.Series(y_val).value_counts().to_dict())

    # Train model
    model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(x_train, y_train, epochs=30, batch_size=32,
              validation_data=(x_val, y_val),
              class_weight=class_weights,
              callbacks=[early_stop],
              verbose=1)

    # Evaluate
    preds = np.argmax(model.predict(x_val), axis=1)
    precision = precision_score(y_val, preds, average='macro', zero_division=0)
    recall = recall_score(y_val, preds, average='macro', zero_division=0)
    accuracy = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro', zero_division=0)

    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    f1_list.append(f1)

# Report average scores
print("\n=== 5-Fold Cross-Validation Summary ===")
print(f"Avg Precision: {np.mean(precision_list):.4f}")
print(f"Avg Recall:    {np.mean(recall_list):.4f}")
print(f"Avg Accuracy:  {np.mean(accuracy_list):.4f}")
print(f"Avg F1 Score:  {np.mean(f1_list):.4f}")

#Testing on Fold 6
train_index = np.concatenate([f[0] for f in train_val_folds])
test_index = test_fold[1]
x_train_raw, x_test_raw = x[train_index], x[test_index]
y_train, y_test = y[train_index], y[test_index]

# Refit scaler only on training data
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train_raw)
x_test = scaler.transform(x_test_raw)

# Reshape for CNN input
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# Class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Train Fold 6 with early stopping
final_model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    min_delta=0.001,
    restore_best_weights=True
)

final_model.fit(x_train, y_train, epochs=30, batch_size=32,
                validation_split=0.1, class_weight=class_weights,
                callbacks=[early_stop], verbose=1)

# Final test evaluation
test_preds = np.argmax(final_model.predict(x_test), axis=1)
test_accuracy = accuracy_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds, average='macro')
test_precision = precision_score(y_test, test_preds, average='macro', zero_division=0)
test_recall = recall_score(y_test, test_preds, average='macro', zero_division=0)

print("\n=== Final Test Fold (Fold 6) Evaluation ===")
print(confusion_matrix(y_test, test_preds))
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1 Score:  {test_f1:.4f}")



=== Fold 1 ===
Class distribution: {0: 51, 1: 49, 2: 46}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.3381 - loss: 1.0986 - val_accuracy: 0.4726 - val_loss: 1.0915
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.4016 - loss: 1.0900 - val_accuracy: 0.5068 - val_loss: 1.0605
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.4931 - loss: 1.0440 - val_accuracy: 0.5342 - val_loss: 0.9472
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5528 - loss: 0.9628 - val_accuracy: 0.5205 - val_loss: 0.9202
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5635 - loss: 0.9367 - val_accuracy: 0.6027 - val_loss: 0.8647
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5889 - loss: 0.9194 - val_accuracy: 0.6301 - val_loss: 0.8515
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6333 - loss: 0.8733 - val_accuracy: 0.6712 - val_loss: 0.7784
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6749 - loss: 0.7946 - val_accuracy: 0.7055 - v

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.3588 - loss: 1.0996 - val_accuracy: 0.3493 - val_loss: 1.0958
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3451 - loss: 1.0988 - val_accuracy: 0.3493 - val_loss: 1.0889
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4020 - loss: 1.0796 - val_accuracy: 0.3767 - val_loss: 1.0563
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4554 - loss: 1.0370 - val_accuracy: 0.6027 - val_loss: 0.9316
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5431 - loss: 0.9814 - val_accuracy: 0.5959 - val_loss: 0.8949
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5362 - loss: 0.9493 - val_accuracy: 0.6438 - val_loss: 0.8715
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5485 - loss: 0.9327 - val_accuracy: 0.5616 - val_loss: 0.8791
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5415 - loss: 0.9225 - val_accuracy: 0.6096 - val_loss: 0.

...

23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.3332 - loss: 1.0972 - val_accuracy: 0.4863 - val_loss: 1.0894
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3869 - loss: 1.0906 - val_accuracy: 0.5411 - val_loss: 1.0700
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4653 - loss: 1.0596 - val_accuracy: 0.4589 - val_loss: 1.0498
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5527 - loss: 0.9818 - val_accuracy: 0.4658 - val_loss: 0.9553
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5522 - loss: 0.9354 - val_accuracy: 0.5342 - val_loss: 0.9298
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5830 - loss: 0.9191 - val_accuracy: 0.5137 - val_loss: 0.9555
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6120 - loss: 0.9039 - val_accuracy: 0.5479 - val_loss: 0.9067
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6031 - loss: 0.8909 - val_accuracy: 0.5274 - val_loss: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.3257 - loss: 1.0985 - val_accuracy: 0.3241 - val_loss: 1.0948
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3866 - loss: 1.0888 - val_accuracy: 0.3586 - val_loss: 1.0689
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4779 - loss: 1.0538 - val_accuracy: 0.5862 - val_loss: 0.9671
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4990 - loss: 0.9997 - val_accuracy: 0.5862 - val_loss: 0.9502
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5767 - loss: 0.9260 - val_accuracy: 0.6069 - val_loss: 0.9170
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5678 - loss: 0.9072 - val_accuracy: 0.6069 - val_loss: 0.9079
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5231 - loss: 0.9499 - val_accuracy: 0.5931 - val_loss: 0.8960
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5914 - loss: 0.8774 - val_accuracy: 0.6276 - val_loss: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


103/103 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.4109 - loss: 1.0672 - val_accuracy: 0.5151 - val_loss: 0.9483
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5701 - loss: 0.9020 - val_accuracy: 0.5808 - val_loss: 0.8673
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6221 - loss: 0.8255 - val_accuracy: 0.6356 - val_loss: 0.7834
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6662 - loss: 0.7421 - val_accuracy: 0.6438 - val_loss: 0.7131
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7394 - loss: 0.6188 - val_accuracy: 0.8164 - val_loss: 0.4358
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8481 - loss: 0.4154 - val_accuracy: 0.9233 - val_loss: 0.2639
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8990 - loss: 0.2810 - val_accuracy: 0.9315 - val_loss: 0.1861
Epoch 8/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9233 - loss: 0.2046 - val_accuracy: 0.9562 - va

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# Set random seed for reproducibility
tf.keras.utils.set_random_seed(492)

# Load and return features and labels
def import_data():
    data = pd.read_csv('mergedshuffledHoldingSquatV2keypoints.csv')
    x = data.iloc[:, :-1].values
    y = data['correct'].astype(int).values
    y = np.searchsorted(np.unique(y), y)  # Make sure y is 0-based index
    return x, y

# Define the CNN-LSTM model
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Main
x, y = import_data()
num_classes = len(np.unique(y))

#6 stratified folds
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
folds = list(skf.split(x, y))
train_val_folds = folds[:5]
test_fold = folds[5]

precision_list, recall_list, accuracy_list, f1_list = [], [], [], []


# Variables to track the best model
best_f1_score = -float('inf')
best_model = None
best_scaler = None
best_fold = 0

# 5-Fold Cross-Validation
for fold_index, (train_index, val_index) in enumerate(train_val_folds, start=1):
    x_train_raw, x_val_raw = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Fit scaler only on training data
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train_raw)
    x_val = scaler.transform(x_val_raw)

    # Reshape for CNN input
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

    # Class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: class_weights[i] for i in range(len(class_weights))}

    print(f"\n=== Fold {fold_index} ===")
    print("Class distribution:", pd.Series(y_val).value_counts().to_dict())

    # Train model
    model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(x_train, y_train, epochs=30, batch_size=32,
              validation_data=(x_val, y_val),
              class_weight=class_weights,
              callbacks=[early_stop],
              verbose=1)

    # Evaluate
    preds = np.argmax(model.predict(x_val), axis=1)
    precision = precision_score(y_val, preds, average='macro', zero_division=0)
    recall = recall_score(y_val, preds, average='macro', zero_division=0)
    accuracy = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro', zero_division=0)

    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    f1_list.append(f1)


    # Save the best model based on F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_model = model
        best_scaler = scaler
        best_fold = fold_index
        # Save the best model and scaler
        model.save('best_model.keras')
        joblib.dump(scaler, 'best_scaler.pkl')
        print(f"Saved best model and scaler from Fold {fold_index} with F1 Score: {f1:.4f}")

# Report average scores
print("\n=== 5-Fold Cross-Validation Summary ===")
print(f"Avg Precision: {np.mean(precision_list):.4f}")
print(f"Avg Recall:    {np.mean(recall_list):.4f}")
print(f"Avg Accuracy:  {np.mean(accuracy_list):.4f}")
print(f"Avg F1 Score:  {np.mean(f1_list):.4f}")

#Testing on Fold 6
train_index = np.concatenate([f[0] for f in train_val_folds])
test_index = test_fold[1]
x_train_raw, x_test_raw = x[train_index], x[test_index]
y_train, y_test = y[train_index], y[test_index]

# Refit scaler only on training data
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train_raw)
x_test = scaler.transform(x_test_raw)

# Reshape for CNN input
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# Class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Train Fold 6 with early stopping
final_model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    min_delta=0.001,
    restore_best_weights=True
)

final_model.fit(x_train, y_train, epochs=30, batch_size=32,
                validation_split=0.1, class_weight=class_weights,
                callbacks=[early_stop], verbose=1)

# Final test evaluation
test_preds = np.argmax(final_model.predict(x_test), axis=1)
test_accuracy = accuracy_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds, average='macro')
test_precision = precision_score(y_test, test_preds, average='macro', zero_division=0)
test_recall = recall_score(y_test, test_preds, average='macro', zero_division=0)

print("\n=== Final Test Fold (Fold 6) Evaluation ===")
print(confusion_matrix(y_test, test_preds))
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1 Score:  {test_f1:.4f}")


print(f"\n=== Best Model Summary ===")
print(f"Best model is from Fold {best_fold} with F1 Score: {best_f1_score:.4f}")


=== Fold 1 ===
Class distribution: {0: 51, 1: 49, 2: 46}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.3381 - loss: 1.0986 - val_accuracy: 0.4726 - val_loss: 1.0915
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4016 - loss: 1.0900 - val_accuracy: 0.5068 - val_loss: 1.0605
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4931 - loss: 1.0440 - val_accuracy: 0.5342 - val_loss: 0.9472
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5528 - loss: 0.9628 - val_accuracy: 0.5205 - val_loss: 0.9202
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5635 - loss: 0.9367 - val_accuracy: 0.6027 - val_loss: 0.8647
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5889 - loss: 0.9194 - val_accuracy: 0.6301 - val_loss: 0.8515
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6333 - loss: 0.8733 - val_accuracy: 0.6712 - val_loss: 0.7784
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6749 - loss: 0.7946 - val_accuracy: 0.7055 - 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.3588 - loss: 1.0996 - val_accuracy: 0.3493 - val_loss: 1.0958
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3451 - loss: 1.0988 - val_accuracy: 0.3493 - val_loss: 1.0889
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4020 - loss: 1.0796 - val_accuracy: 0.3767 - val_loss: 1.0563
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.4554 - loss: 1.0370 - val_accuracy: 0.6027 - val_loss: 0.9316
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5431 - loss: 0.9814 - val_accuracy: 0.5959 - val_loss: 0.8949
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.5362 - loss: 0.9493 - val_accuracy: 0.6438 - val_loss: 0.8715
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.5485 - loss: 0.9327 - val_accuracy: 0.5616 - val_loss: 0.8791
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5415 - loss: 0.9225 - val_accuracy: 0.6096 - val_loss: 0.

...

23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.3332 - loss: 1.0972 - val_accuracy: 0.4863 - val_loss: 1.0894
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.3869 - loss: 1.0906 - val_accuracy: 0.5411 - val_loss: 1.0700
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4653 - loss: 1.0596 - val_accuracy: 0.4589 - val_loss: 1.0498
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5527 - loss: 0.9818 - val_accuracy: 0.4658 - val_loss: 0.9553
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5522 - loss: 0.9354 - val_accuracy: 0.5342 - val_loss: 0.9298
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5830 - loss: 0.9191 - val_accuracy: 0.5137 - val_loss: 0.9555
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6120 - loss: 0.9039 - val_accuracy: 0.5479 - val_loss: 0.9067
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6031 - loss: 0.8909 - val_accuracy: 0.5274 - val_loss: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.3257 - loss: 1.0985 - val_accuracy: 0.3241 - val_loss: 1.0948
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3866 - loss: 1.0888 - val_accuracy: 0.3586 - val_loss: 1.0689
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4779 - loss: 1.0538 - val_accuracy: 0.5862 - val_loss: 0.9671
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4990 - loss: 0.9997 - val_accuracy: 0.5862 - val_loss: 0.9502
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5767 - loss: 0.9260 - val_accuracy: 0.6069 - val_loss: 0.9170
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5678 - loss: 0.9072 - val_accuracy: 0.6069 - val_loss: 0.9079
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5231 - loss: 0.9499 - val_accuracy: 0.5931 - val_loss: 0.8960
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5914 - loss: 0.8774 - val_accuracy: 0.6276 - val_loss: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


103/103 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.4109 - loss: 1.0672 - val_accuracy: 0.5151 - val_loss: 0.9483
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5701 - loss: 0.9020 - val_accuracy: 0.5808 - val_loss: 0.8673
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6221 - loss: 0.8255 - val_accuracy: 0.6356 - val_loss: 0.7834
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6662 - loss: 0.7421 - val_accuracy: 0.6438 - val_loss: 0.7131
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7394 - loss: 0.6188 - val_accuracy: 0.8164 - val_loss: 0.4358
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8481 - loss: 0.4154 - val_accuracy: 0.9233 - val_loss: 0.2639
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8990 - loss: 0.2810 - val_accuracy: 0.9315 - val_loss: 0.1861
Epoch 8/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9233 - loss: 0.2046 - val_accuracy: 0.956

(3) Training by using CNN-LSTM via keypoints + augmetation

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Set random seed for reproducibility
tf.keras.utils.set_random_seed(492)

# Load data
def import_data():
    data = pd.read_csv('mergedshuffledHoldingSquatV2keypoints.csv')
    x = data.iloc[:, :-1].values
    y = data['correct'].astype(int).values
    y = np.searchsorted(np.unique(y), y)
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x)
    return x, y

# Balance dataset using SMOTE
def balance_data(x, y):
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    x_resampled, y_resampled = smote.fit_resample(x, y)
    print("Balanced class distribution:")
    print(pd.Series(y_resampled).value_counts())
    return x_resampled, y_resampled

# Add Gaussian noise
def augment_data(x, noise_factor=0.05):
    noise = np.random.normal(loc=0.0, scale=noise_factor, size=x.shape)
    return x + noise

# Model definition
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Main
x, y = import_data()
x, y = balance_data(x, y)
num_classes = len(np.unique(y))

# Create 6 stratified folds
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
folds = list(skf.split(x, y))
train_val_folds = folds[:5]
test_fold = folds[5]

# best model
best_model_path = "best_cnn_lstm_model.keras"
best_fold, best_f1 = None, -1
precision_list, recall_list, accuracy_list, f1_list = [], [], [], []

# Perform 5-Fold
for fold_index, (train_index, val_index) in enumerate(train_val_folds, start=1):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Augment training data
    x_train = augment_data(x_train, noise_factor=0.05)

    # Reshape
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

    # Class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: class_weights[i] for i in range(len(class_weights))}

    print(f"\n=== Fold {fold_index} ===")
    print("Class distribution:", pd.Series(y_val).value_counts().to_dict())

    # Model & Callbacks
    model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        x_train, y_train,
        epochs=30,
        batch_size=32,
        validation_data=(x_val, y_val),
        class_weight=class_weights,
        callbacks=[early_stop],
        verbose=1
    )

    # Evaluate
    preds = np.argmax(model.predict(x_val), axis=1)
    precision = precision_score(y_val, preds, average='macro', zero_division=0)
    recall = recall_score(y_val, preds, average='macro', zero_division=0)
    accuracy = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro', zero_division=0)

    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    # Save metrics
    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    f1_list.append(f1)

    if f1 > best_f1:
        best_f1 = f1
        best_fold = fold_index
        model.save(best_model_path)

# Print summary
print("\n=== 5-Fold Validation Summary ===")
print(f"Avg Precision: {np.mean(precision_list):.4f}")
print(f"Avg Recall:    {np.mean(recall_list):.4f}")
print(f"Avg Accuracy:  {np.mean(accuracy_list):.4f}")
print(f"Avg F1 Score:  {np.mean(f1_list):.4f}")
print(f"Best Fold: {best_fold} with F1 Score: {best_f1:.4f}")
print(f"Best model saved at: {best_model_path}")

# === Final Test Fold Evaluation ===
test_index = test_fold[1]
train_index = np.concatenate([f[0] for f in train_val_folds])
x_train, x_test = x[train_index], x[test_index]
y_train, y_test = y[train_index], y[test_index]

# Augment and reshape
x_train = augment_data(x_train, noise_factor=0.05)
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# Class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Final model training on Fold 6
final_model = create_cnn_lstm_model(input_shape=(x_train.shape[1], x_train.shape[2]), num_classes=num_classes)
early_stop = EarlyStopping(monitor='val_accuracy', patience=2, min_delta=0.001, restore_best_weights=True)

final_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=30,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate on final test set
test_preds = np.argmax(final_model.predict(x_test), axis=1)
test_accuracy = accuracy_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds, average='macro')
test_precision = precision_score(y_test, test_preds, average='macro', zero_division=0)
test_recall = recall_score(y_test, test_preds, average='macro', zero_division=0)

print("\n=== Final Test Fold (Fold 6) Evaluation ===")
print(confusion_matrix(y_test, test_preds))
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1 Score:  {test_f1:.4f}")


Balanced class distribution:
1    305
2    305
0    305
Name: count, dtype: int64

=== Fold 1 ===
Class distribution: {1: 51, 2: 51, 0: 51}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - accuracy: 0.3043 - loss: 1.0987 - val_accuracy: 0.5033 - val_loss: 1.0902
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.4384 - loss: 1.0854 - val_accuracy: 0.4837 - val_loss: 1.0424
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5187 - loss: 1.0086 - val_accuracy: 0.5556 - val_loss: 0.9509
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5629 - loss: 0.9325 - val_accuracy: 0.5882 - val_loss: 0.9289
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5732 - loss: 0.9215 - val_accuracy: 0.5425 - val_loss: 0.9302
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5959 - loss: 0.9042 - val_accuracy: 0.5490 - val_loss: 0.9354
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5929 - loss: 0.9022 - val_accuracy: 0.5948 - val_loss: 0.9142
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5984 - loss: 0.8728 - val_accuracy: 0.6013 - 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - accuracy: 0.3208 - loss: 1.0999 - val_accuracy: 0.5098 - val_loss: 1.0901
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4086 - loss: 1.0909 - val_accuracy: 0.3922 - val_loss: 1.0674
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4525 - loss: 1.0578 - val_accuracy: 0.6144 - val_loss: 0.9601
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4568 - loss: 1.0389 - val_accuracy: 0.5817 - val_loss: 0.9445
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5544 - loss: 0.9476 - val_accuracy: 0.6340 - val_loss: 0.9198
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5287 - loss: 0.9691 - val_accuracy: 0.6013 - val_loss: 0.8743
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5474 - loss: 0.9104 - val_accuracy: 0.5621 - val_loss: 0.8838
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5494 - loss: 0.9309 - val_accuracy: 0.6275 - val_loss: 0.

...

24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.3348 - loss: 1.0962 - val_accuracy: 0.4803 - val_loss: 1.0825
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4279 - loss: 1.0780 - val_accuracy: 0.5592 - val_loss: 1.0155
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.4803 - loss: 1.0246 - val_accuracy: 0.5132 - val_loss: 0.9278
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5284 - loss: 0.9792 - val_accuracy: 0.5789 - val_loss: 0.8839
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5500 - loss: 0.9225 - val_accuracy: 0.5658 - val_loss: 0.8992
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5542 - loss: 0.9097 - val_accuracy: 0.6053 - val_loss: 0.8275
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5835 - loss: 0.8869 - val_accuracy: 0.6316 - val_loss: 0.8418
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5953 - loss: 0.8602 - val_accuracy: 0.6250 - val_loss: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.3133 - loss: 1.1004 - val_accuracy: 0.3816 - val_loss: 1.0952
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3859 - loss: 1.0897 - val_accuracy: 0.4145 - val_loss: 1.0847
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.4665 - loss: 1.0586 - val_accuracy: 0.5066 - val_loss: 1.0105
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5320 - loss: 0.9539 - val_accuracy: 0.6382 - val_loss: 0.9456
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5856 - loss: 0.9057 - val_accuracy: 0.6316 - val_loss: 0.9050
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5844 - loss: 0.9105 - val_accuracy: 0.5724 - val_loss: 0.8832
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5711 - loss: 0.8948 - val_accuracy: 0.5789 - val_loss: 0.8618
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5983 - loss: 0.8602 - val_accuracy: 0.6184 - val_loss: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


108/108 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.3952 - loss: 1.0709 - val_accuracy: 0.5602 - val_loss: 0.9125
Epoch 2/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5756 - loss: 0.8995 - val_accuracy: 0.6204 - val_loss: 0.8171
Epoch 3/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6220 - loss: 0.8157 - val_accuracy: 0.7461 - val_loss: 0.6501
Epoch 4/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7532 - loss: 0.5979 - val_accuracy: 0.8351 - val_loss: 0.4182
Epoch 5/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8041 - loss: 0.5020 - val_accuracy: 0.8691 - val_loss: 0.3509
Epoch 6/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8594 - loss: 0.3755 - val_accuracy: 0.8770 - val_loss: 0.3142
Epoch 7/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8890 - loss: 0.3015 - val_accuracy: 0.9372 - val_loss: 0.2052
Epoch 8/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9134 - loss: 0.2366 - val_accuracy: 0.9450 - 